In [ ]:
import json

gender_map = {
    "m sg": "陽性單數",
    "f sg": "陰性單數",
    "n sg": "中性單數",
    "m pl": "陽性複數",
    "f pl": "陰性複數",
    "n pl": "中性複數",
    "pl": "複數"
}

def convert_gender_number(gn):
    return gender_map.get(gn, gn)

def convert_case(case):
    case_map = {
        "Akkusativ": "第四格",
        "Dativ": "第三格",
        "Genitiv": "第二格"
    }
    return case_map.get(case, case)

def write_components(components, md_lines):
    subject = components.get("subject")
    if subject:
        md_lines.append(f"   - 主語：{subject['de']}（{subject['zh']}）")
        if "gender_number" in subject:
            md_lines.append(f"       * 性質：{convert_gender_number(subject['gender_number'])}")
        if subject.get("original_nominative_form") and subject['de'] != subject['original_nominative_form']:
            md_lines.append(f"       * 原形：{subject['original_nominative_form']}")

    if "verb_form" in components:
        md_lines.append(f"   - 動詞形態：{components['verb_form']}")

    for obj in components.get("objects", []):
        line = f"   - 受詞：{obj['de']}（{obj['zh']}, {convert_case(obj['case'])}）"
        md_lines.append(line)
        if "gender_number" in obj:
            md_lines.append(f"       * 性質：{convert_gender_number(obj['gender_number'])}")
        if obj.get("original_nominative_form"):
            md_lines.append(f"       * 原形：{obj['original_nominative_form']}")
    md_lines.append("")

def json_to_md(json_file, output_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    md_lines = []
    md_lines.append("# **判決中的動詞整理（含介詞支配）** {#verbs-in-judgments}\n\n")

    for vp in data["verb_phrases"]:
        lemma = vp["lemma"]
        meanings = vp.get("zh_meanings", {})
        grammar = vp.get("grammar", {})

        md_lines.append(f"## {lemma}\n")
        md_lines.append("- **中文含義**：")
        if "original" in meanings:
            md_lines.append(f"  - 原意：{', '.join(meanings['original'])}")
        if "contextual" in meanings:
            md_lines.append(f"  - 語境意義：{meanings['contextual']}")
        if grammar.get("case_requirement"):
            md_lines.append(f"- **受詞格位需求**：{convert_case(grammar['case_requirement'])}")
        if grammar.get("is_reflexive"):
            md_lines.append("- **特性**：反身動詞")
        if grammar.get("is_modal"):
            md_lines.append("- **特性**：情態動詞")
        if grammar.get("is_separable_verb"):
            md_lines.append("- **特性**：分離動詞")
        if grammar.get("gov_preposition"):
            md_lines.append(f"- **主要支配介詞**：`{grammar['gov_preposition']}`")
        md_lines.append("")

        # 一般例句
        examples = vp.get("examples", [])
        if examples:
            md_lines.append("### 一般例句\n")
            for idx, ex in enumerate(examples, 1):
                md_lines.append(f"{idx}. **德文例句**：`{ex['de']}`")
                md_lines.append(f"   - 中文翻譯：{ex['zh']}")
                write_components(ex.get("components", {}), md_lines)

        # 介詞支配例句
        preps = vp.get("prepositional_phrases", [])
        if preps:
            md_lines.append("### 介詞支配與搭配\n")
            for prep in preps:
                prep_case = convert_case(prep.get("case", ""))
                md_lines.append(f"- **支配介詞結構**：`{prep['preposition']} + {prep_case}`")
                example = prep.get("example")
                if example:
                    md_lines.append(f"  - 例句：`{example['de']}`")
                    md_lines.append(f"    - 翻譯：{example['zh']}")
                    write_components(example.get("components", {}), md_lines)

        md_lines.append("\n***\n")

    # metadata
    meta = data.get("metadata", {})
    md_lines.append(f"> **資料來源**：{meta.get('source', '不明')}\n")
    md_lines.append(f"> **語言版本**：{meta.get('language', '不明')}\n")
    md_lines.append(f"> **處理時間**：{meta.get('processed_date', '不明')}\n")
    md_lines.append(f"> **版本**：{meta.get('version', '未知')}\n")

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("\n".join(md_lines))

In [27]:
 

# 使用範例
# json_to_md('german_verbs_tw.json', 'output_verbs_tw.md')


# 使用範例
json_to_md('german_verbs_tw_02.json', 'output_verbs_tw_02.md')